In [1]:
using Pkg
Pkg.activate(@__DIR__);
Pkg.instantiate()

In [2]:
using CSV
using DataFrames
using Symbolics
using SparseArrays
using StaticArrays
using LinearAlgebra
using ForwardDiff

In [3]:
df = CSV.read("processed_data/merged_2021-02-03-16-55-55_seg_2.csv", DataFrame)
# names(df)

p = Matrix(df[!, ["pos x","pos y","pos z"]])
v = Matrix(df[!, ["vel x","vel y","vel z"]])
v̇ = Matrix(df[!, ["acc x","acc y","acc z"]])
ω = Matrix(df[!, ["ang vel x","ang vel y","ang vel z"]])
ω̇ = Matrix(df[!, ["ang acc x","ang acc y","ang acc z"]])
Ω = Matrix(df[!, ["mot 1","mot 2","mot 3","mot 4"]])
Ω̇ = Matrix(df[!, ["dmot 1","dmot 2","dmot 3","dmot 4"]]);

𝕏𝑓 = hcat(v, ω, Ω);
𝕏𝑡 = hcat(v, ω, ω̇, Ω, Ω̇);

size(𝕏𝑡)

(12363, 17)

In [21]:
3 * (14*2 + 1)

87

In [22]:
14*2 + 1

29

In [49]:
function residual(
    x::MVector{17},
    β::MVector{93},
)
    v, ω, ω̇, Ω, Ω̇ = x[1:3], x[4:6], x[7:9], x[10:13], x[14:17]

    # D ∈ ℝ³ˣ²¹¹ 
    D = reshape(β[1:87], 3, 29)
    J₁₁, J₁₂, J₁₃, J₂₂, J₂₃, J₃₃ = β[end-5:end]
    𝕁 = [J₁₁  J₁₂  J₁₃;
         J₁₂  J₂₂  J₂₃;
         J₁₃  J₂₃  J₃₃];

    # Body torque
    τ = 𝕁 * ω̇ + ω × (𝕁 * ω)

    # Quadratic model
    y = [v; ω; Ω; Ω̇]
    # z = [1; y; kron(y, y)]
    z = [1; y; y.*y]

    τ̂ = D * z

    r = τ - τ̂
    return r
end

function residual_vec(X::Matrix, β::MVector{93, T}) where {T}
    N = size(X)[1]
    X = MMatrix{N, 17}(X)
    𝐫 = zeros(T, 3, N)

    for i in 1:N
        𝐫[:, i] .= residual(X[i,:], β)
        # 𝐫[:, i] .= residual(MVector{17}(X[i,:]), β)
    end

    return vec(𝐫)
end

function cost(X, β)
    𝐫 = residual_vec(X, β) 
    S = 𝐫' * 𝐫
    return S
end

cost (generic function with 1 method)

In [50]:
size(𝕏𝑡)

(12363, 17)

In [51]:
β = @MVector rand(93)
𝐫 = residual_vec(𝕏𝑡[1:10:end,:], β)

In [31]:
𝕏𝑡[1:1000:end,:]

13×17 Matrix{Float64}:
 -0.0220132  -0.0505215  -0.00238934  …     28.2541   -753.627    -79.0421
  0.706389    0.082822    0.00133852      -760.802     788.097    371.144
  1.34686    -0.207984    0.376801        -638.4     -1194.51     367.391
  1.26385    -2.24899     0.774266       -1157.0      -761.463  -1137.05
 -1.60732     1.25156     2.47652          -87.5965  -2433.27   -2931.24
 -1.09119    -1.80517     3.80418     …    105.809   -1999.12   -1925.48
 -2.67725     5.70941     0.633674        -329.01     1876.27    3833.92
  1.64937    -2.94585     1.53701        -2295.7     -1595.02   -1050.79
  1.96375     0.924174   -1.27693        -2426.68      668.464   5266.45
  1.03264    -0.443507   -0.997216        2563.94    -1078.51    2420.15
  0.292178    2.72825     0.10056     …   -102.376     773.502    247.756
  0.382385    1.39379     0.085851          67.4603   -807.286   -129.425
  0.253844    0.0313993  -0.0107087        198.806     464.724     80.0589

In [32]:
ForwardDiff.jacobian(_β->residual_vec(𝕏𝑡[1:1000:end,:], _β), β)

LoadError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{var"#1#2", Float64}, Float64, 93})
[0mClosest candidates are:
[0m  (::Type{T})(::Real, [91m::RoundingMode[39m) where T<:AbstractFloat at rounding.jl:200
[0m  (::Type{T})(::T) where T<:Number at boot.jl:760
[0m  (::Type{T})([91m::AbstractChar[39m) where T<:Union{AbstractChar, Number} at char.jl:50
[0m  ...

In [29]:
residual_vec(𝕏𝑡[1:1000:end,:], β)

39-element Vector{Float64}:
 -2.391641704355393e6
 -3.293338929697577e6
 -3.3024152976426794e6
 -2.516777300634501e6
 -3.5390422077605906e6
 -3.777383434124063e6
 -3.827458777198678e6
 -4.204798543450791e6
 -6.517294443026971e6
 -3.6831163584902324e6
 -4.498992245201497e6
 -4.795546299034991e6
 -1.2096313909384664e7
  ⋮
 -1.057387341930676e7
 -1.410614853985585e7
 -1.2570222628983347e7
 -2.801154591712602e6
 -3.733042552253871e6
 -3.914324077995674e6
 -2.432317738674514e6
 -3.3576010122908163e6
 -3.4702531776609328e6
 -2.419482776945394e6
 -3.2027514965958367e6
 -3.174715271747858e6

In [12]:
using CSV
using DataFrames
using StaticArrays
export LinearAlgebra: ×, norm

In [15]:
df = CSV.read("processed_data/merged_2021-02-03-16-55-55_seg_2.csv", DataFrame)
# names(df)

p = Matrix(df[!, ["pos x","pos y","pos z"]])
v = Matrix(df[!, ["vel x","vel y","vel z"]])
v̇ = Matrix(df[!, ["acc x","acc y","acc z"]])
ω = Matrix(df[!, ["ang vel x","ang vel y","ang vel z"]])
ω̇ = Matrix(df[!, ["ang acc x","ang acc y","ang acc z"]])
Ω = Matrix(df[!, ["mot 1","mot 2","mot 3","mot 4"]])
Ω̇ = Matrix(df[!, ["dmot 1","dmot 2","dmot 3","dmot 4"]]);

𝕏𝑓 = hcat(v, ω, Ω);
𝕏𝑡 = hcat(v, ω, Ω, Ω̇);

In [20]:
𝕏𝑓 = hcat(v, ω, Ω);
𝕏𝑡 = hcat(v, ω, Ω, Ω̇);

In [21]:
size(𝕏𝑡)

(12363, 14)

In [ ]:
function quad_moments(
                      x::SArray{6},
                    #   ω::SArray{3}, 
                    #   ω̇::SArray{3}, 
                      J::SArray{3,3}
                      )
    ω, ω̇ = x[1:3], x[4:6]

    τ = J * ω̇ + ω × (J * ω)
    return τ
end

function cost_function(x::SArray{6})
    norm( - x)
end

function residual(β::SArray{20}, xᵢ::SArray{14}, )

    J₁₁, J₁₂, J₁₃, J₂₂, J₂₃, J₃₃ = β[]

end

In [ ]:
plt = plot3d(
    1,
    xlim = (-30, 30),
    ylim = (-30, 30),
    zlim = (0, 60),
    title = "Lorenz Attractor",
    marker = 2,
)

# build an animated gif by pushing new points to the plot, saving every 10th frame
@gif for i=1:1500
    step!(attractor)
    push!(plt, attractor.x, attractor.y, attractor.z)
end every 10

In [47]:
?plot3d!

search: plot3d! plot3d



No documentation found.

`Plots.plot3d!` is a `Function`.

```
# 1 method for generic function "plot3d!":
[1] plot3d!(args...; kw...) in Plots at /Users/AlexanderBouman/.julia/packages/Plots/qbc7U/src/shorthands.jl:410
```


In [56]:
ExampleInputDF